# Processing Pairwise Force Reconstruction (PFR) Files from GROMACS-FDA

The FDA gmx patch writes out pfr files that are lists of residue pairs with their respective pairwise forces for different simulation frames (see https://github.com/HITS-MBM/gromacs-fda). We convert those to numpy arrays.

In [1]:
import numpy as np
import os
import pickle
import mdtraj as md
from tqdm.notebook import trange, tqdm
from scipy import stats
from timeit import default_timer as timer
import pandas as pd
import pytraj as pt
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 200

A dict PAIRWISE_FORCES is used to organize collected data and provide initial file paths to the .pfr files that are processed here

In [2]:
with open ('PAIRWISE_FORCES.pkl', 'rb') as fp:
    PAIRWISE_FORCES = pickle.load(fp)

For each runtype PAIRWISE_FORCES will contain:

| Key | Description |
|:----|:-------------|
| `folder` | Paths to PFR files per run |
| `pf_vec` | Vector-valued pairwise force files |
| `F_t` | Time-series force matrices |


# Load and convert pairwise interactions

Assumes access to raw .pfr files, which are not included in this repository due to filesize

In [6]:
n_res = md.load("protein.pdb").n_residues
#n_res = 616

def list_to_force_matrix(frame):
    pairwise_forces = np.empty((n_res, n_res))
    pairwise_forces[:] =  np.nan

    pairwise_forces[frame[:,0].astype(int), frame[:,1].astype(int)] = frame[:,2]
    pairwise_forces[frame[:,1].astype(int), frame[:,0].astype(int)] = frame[:,2]
    return pairwise_forces

def pfr_file_to_matrix_list(file):
    asnpy = file.replace("pfr","npy")
    force_t_npy_fname = file.replace(".pfr", "F_t.npy")
    ## check whether pfr file was already processed
    if not os.path.exists(force_t_npy_fname):
        start = timer()
        f = np.genfromtxt(file, skip_header=1, comments="frame")
        end = timer()
        print(end-start, f"seconds to load text file {file}")
    else:
        print(f"psr file {file} already processed")
        return np.load(force_t_npy_fname)
    
    
    
    start=timer()
    np.save(file.replace("pfr","npy"), f)
    ## extract indices corresponding to a new frame in the pfr file
    framejump = []
    for i in range(1, len(f)):
        if f[i,0] < f[i-1,0]:
            framejump.append(i)

    split_into_frames = [f[framejump[i-1]:framejump[i]] for i in range(1,len(framejump))]

    F_t = np.array([list_to_force_matrix(frame) for frame in split_into_frames])
    np.save(force_t_npy_fname, F_t)
    end = timer()
    print(end-start, f"seconds to process forces matrix")
    return F_t

In [ ]:
for key in tqdm([PAIRWISE_FORCES], desc="Run type"):
    for file in tqdm(PAIRWISE_FORCES[key]["folder"], desc="pfr file", leave=False):
        PAIRWISE_FORCES[key]["F_t"] = pfr_file_to_matrix_list(file)
                
    start = timer();
    ## load in F_ij(t) matrix for all replicas, concatenate them and form the temporal mean/standard dev
    F_t = np.concatenate([np.load(f.replace(".pfr", "F_t.npy"),
                             mmap_mode='c') # lets us treat a file on disk as if it were all in memory
                  for f in tqdm(PAIRWISE_FORCES[key]["folder"], leave=False, desc="np.load(F_t)")])
    print(key, F_t.shape)
    PAIRWISE_FORCES[key]["F_t_mean"] = np.nanmean(F_t, axis=0)
    PAIRWISE_FORCES[key]["F_t_std"] = np.nanstd(F_t, axis=0)
    print(key, PAIRWISE_FORCES[key]["F_t_mean"].shape)
    del F_t

    end = timer(); print(end-start, f"seconds for time averaging")

# Vector-valued pairwise forces

Read in vector pfr files and calculate residue-wise total instantaneous force

In [75]:
## we ignore forces between residues with small sequence distance
## as we are primarily interested in inter-domain interactions
n_ignore_neighbors=25

In [6]:
def pfr_to_residuewise_total_Force_t(key, i):
        topfolder = "/".join((PAIRWISE_FORCES[key]["folder"][0]).split("/")[:-1])
        traj = pt.iterload(topfolder + f"/{i+1}/traj_comp.xtc",
                           topfolder + f"/{i+1}/step5_input.pdb", stride=500) ## loaded just for the top
        start = timer()
        file = PAIRWISE_FORCES[key]["pf_vec"][i]
        df = pd.read_csv(file, 
                         delim_whitespace=True, comment="f", 
                         skiprows=1, header=None, usecols=range(5))
        end = timer(); print(end-start, f"seconds to load text file {file}")
        
        fi = df[0].values
        start = timer()
        framejump = [i for i in range(len(fi)) if fi[i] < fi[i-1]]
        np.save(f"framejump_{key}_{i}", framejump)
        end = timer(); print(end-start, f"seconds to determine where frames start and end")
        
        ## extract total instantaneous force by residue
        start = timer()
        residuewise_totalforce = []
        for res in tqdm(range(len(traj.top.select("@CA"))), leave=True):
            ## extract forces acting on selected residue
            to = df.loc[(df[0]==res)][[2,3,4]]
            fro = df.loc[(df[1]==res)][[2,3,4]]
            fro[[2,3,4]] = -1*fro[[2,3,4]] # this edits the copy "fro", not the original df
            df_sel = pd.concat([to,fro], axis=0)[[2,3,4]]

            ## sum force vectors over bins corresponding to time intervals determined in framejump
            ## i.e. the resulting array contains all timesteps
            summed_forces = stats.binned_statistic(df_sel.index.values, 
                                   df_sel.values.T, 
                                   'sum', bins=framejump
                                  ).statistic.T
            residuewise_totalforce.append(summed_forces)
        np.save(f"residuewise_totalforce_vectors_{key}_{i}", residuewise_totalforce)
        end = timer(); print(end-start, f"seconds to calculate residue-wise total force")
        return np.array(residuewise_totalforce)

In [11]:
def pfr_to_residuewise_total_Force_t_with_neighborcutoff(key, i, n_ignore_neighbors=25):
        topfolder = "/".join((PAIRWISE_FORCES[key]["folder"][0]).split("/")[:-1])
        traj = pt.iterload(topfolder + f"/{i+1}/traj_comp.xtc",
                           topfolder + f"/{i+1}/step5_input.pdb", stride=500) ## loaded just for the top
        start = timer()
        file = PAIRWISE_FORCES[key]["pf_vec"][i]
        df = pd.read_csv(file, 
                         delim_whitespace=True, comment="f", 
                         skiprows=1, header=None, usecols=range(5))
        end = timer(); print(end-start, f"seconds to load text file {file}")
        
        fi = df[0].values ## list noting which frame each df entry corresponds to
        
        ## figure out which lines correspond to which frame
        if os.path.exists(f"framejump_{key}_{i}.npy"):
            print(f"loading previously processed framejump file framejump_{key}_{i}.npy")
            framejump = np.load(f"framejump_{key}_{i}.npy")
        else:
            start = timer()
            framejump = [i for i in range(len(fi)) if fi[i] < fi[i-1]]
            np.save(f"framejump_{key}_{i}", framejump)
            end = timer(); print(end-start, f"seconds to determine where frames start and end")
        
        ## extract total instantaneous force by residue
        start = timer()
        residuewise_totalforce = []
        for res in tqdm(range(len(traj.top.select("@CA"))), leave=True):
            ## ignore forces between sequence-near residues
            df_res = df.loc[(df[0]==res)]
            df_longrange = df_res.loc[abs(df_res[0]-df_res[1]) >= n_ignore_neighbors]
            ## extract forces acting on selected residue
            to = df_longrange[[2,3,4]]
            ## ...and other way round
            df_res = df.loc[(df[1]==res)]
            df_longrange = df_res.loc[abs(df_res[0]-df_res[1]) >= n_ignore_neighbors]
            fro = -df_longrange[[2,3,4]]
            # fro = df_longrange[[2,3,4]]
            # fro[[2,3,4]] = -1*fro[[2,3,4]] 
            df_sel = pd.concat([to,fro], axis=0)[[2,3,4]]
            ## treat empty df_sel
            if len(df_sel) == 0:
                df_sel  = pd.DataFrame([[0,0,0]])
            ## sum force vectors over bins corresponding to time intervals determined in framejump
            ## i.e. the resulting array contains all timesteps
            summed_forces = stats.binned_statistic(df_sel.index.values, 
                                   df_sel.values.T, 
                                   'sum', bins=framejump
                                  ).statistic.T
            residuewise_totalforce.append(summed_forces)
        np.save(f"residuewise_totalforce_vectors_neighborcut_{key}_{i}", residuewise_totalforce)
        end = timer(); print(end-start, f"seconds to calculate residue-wise total force")
        return np.array(residuewise_totalforce)

In [ ]:
%%time
## perform processing for each setup and replica
for key in tqdm(PAIRWISE_FORCES):
    for i in tqdm(range(3), leave=False):
        pfr_to_residuewise_total_Force_t_with_neighborcutoff(key, i)